In [43]:
from langchain_community.document_loaders import PyPDFLoader
import tqdm
import os
from dotenv import load_dotenv
load_dotenv()

True

In [73]:
loader = PyPDFLoader('files\\result.pdf')
documents = loader.load()

In [31]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

OPENAI

In [33]:
os.environ['OPENAI-KEY'] = os.getenv('OPENAI_KEY')
openai = ChatOpenAI(model='gpt-3.5-turbo', api_key=os.environ['OPENAI-KEY'])

HuggingFace 

GROQ LLAMA-3

In [45]:
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_KEY')
groq_model = ChatGroq(name='llama3-8b-8192', api_key=os.environ['GROQ_API_KEY'])

Splitting the document

In [74]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
docs = text_splitter.split_documents(documents)

In [35]:
# Embeddings model 
embedding = HuggingFaceEmbeddings(model_name = 'BAAI/bge-base-en-v1.5')

CHROMA VECTOR DB

In [75]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents = docs, embedding=embedding)

In [67]:
from langchain_core.prompts import ChatPromptTemplate
receipt_order_prompt = ChatPromptTemplate.from_template("""
Aanswer the following question based on only the following context. 
Give consise answer to the query and think step by step before giving a detailed answer. 
Don't prepend unnecessary information on your own and just answer what has been asked.
<context> {context} </context> Question : {input}
""")
result_prompt = ChatPromptTemplate.from_template("""
Aanswer the following question based on only the following context.
Don't prepend unnecessary information on your own and just answer what has been asked.
<context> {context} </context> Question : {input}
""")

In [21]:
## Context is autofilled with the help of chains 
# chains are sequence of calls to llm, tool or preprocessing step 

In [76]:
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm=groq_model, prompt=result_prompt)
# this chain will put the context retrieved from the model into the prompt and the model will generate a response from it

In [77]:
from langchain.chains import create_retrieval_chain
# This retriver is an interface to the vectorstore. It retrieves the documents from the document vector store using a similarity function and apends it into the prompt
retriever = vectorstore.as_retriever() # kwargs = {'k' : top_k} 
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001C6636A7AF0>)

In [70]:
agent = create_retrieval_chain(retriever, document_chain)

In [78]:
response = agent.invoke({
    'input' : 'What is the Total Income/Revenue that the company made in the latest quarter in comparison with the last quarter. What is its Net Proft before or after tax. '
})

In [79]:
print(response['answer'])

The company made a total income of Rs. 685.00 lakhs in the latest quarter (Half Year Ended 31-Mar-24) compared to Rs. 836.70 lakhs in the last quarter (Half Year Ended 30-Sep-23).

The net profit before tax for the latest quarter is Rs. 37.63 lakhs and the net profit after tax is Rs. 26.81 lakhs.


**Actor and Critic Agent Architecture** 

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['groq_key'] = os.getenv('GROQ_KEY')

In [22]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from typing import Annotated, Sequence, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
embedding = HuggingFaceEmbeddings(model_name = 'BAAI/bge-base-en-v1.5')

d:\bse-orderly\virtual_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader('sample.pdf')
document = loader.load()

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
docs = text_splitter.split_documents(document)

In [12]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(documents = docs, embedding=embedding)

In [18]:
from langchain.tools.retriever import create_retriever_tool

## To use a RAG model in Actor-Critic model, we need to build a retriever tool
retriever = vectorstore.as_retriever()
retriever_tool = create_retriever_tool(
    retriever, 
    name="Financial_Info_Retriver", 
    description = "Retrieve information from the provided financial documents like quarterly financial results and order receipt."
)
tools = [retriever_tool]

In [21]:
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages : Annotated[Sequence[BaseMessage], add_messages]

In [ ]:
from langchain_core.prompts import PromptTemplate

def grader(state : AgentState) -> Literal["generate", "rewrite"]:
    llm = ChatGroq(model='llama3-8b-8192', api_key=os.environ['groq_key'], streaming=True)
    llm_with_tool = llm.with_stru

    

